In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import KQA_Dataset_BG_BERTmul as kqd
import KQA_Model_BERTmul as kqm
import sentencepiece as spm
import torch
import torch.nn as nn
# from torch.nn.parallel import DistributedDataParallel
# from apex.parallel import DistributedDataParallel as "DDP
import warnings
warnings.filterwarnings(action='ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("GPU count {}".format(torch.cuda.device_count()))
print ("GPU for Training : {} ".format(device))

GPU count 1
GPU for Training : cuda 


In [2]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
KorQuAD_Model= kqm.QuestionAnswering()
# KorQuAD_Model = nn.DataParallel(KorQuAD_Model)
KorQuAD_Model.to(device)
print("Model Load")

Model Load


In [4]:
KorQuAD_Model

QuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [5]:
train_data, test_data = kqd.load_and_cache_dataset(input_file="./KorQuAD_v1.0_train.json",
                                           max_seq_length=512,
                                           testset_ratio=0.05)

In [6]:
count_trainable_parameters(KorQuAD_Model)

177854978

In [7]:
len(train_data)

53471

In [8]:
len(test_data)

2815

In [9]:
from torch.utils.data import DataLoader
batch_size=8
learning_rate = 5e-5

train_dataloader = DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=4)
test_dataloader = DataLoader(test_data,batch_size=batch_size,shuffle=True,num_workers=4)

In [ ]:
for epoch in range(20):
    train_meanloss=0
    for i,(input_ids, segment_ids, input_mask, start_positions, end_positions) in enumerate(train_dataloader):
        input_ids = torch.tensor(input_ids,dtype=torch.long)
        input_ids = input_ids.to(device)
        input_mask = torch.tensor(input_mask,dtype=torch.long)
        input_mask = input_mask.to(device)
        segment_ids = torch.tensor(segment_ids,dtype=torch.long)
        segment_ids = segment_ids.to(device)
        start_positions = torch.tensor(start_positions,dtype=torch.long)
        start_positions = start_positions.to(device)
        end_positions = torch.tensor(end_positions,dtype=torch.long)
        end_positions = end_positions.to(device)
        #KorQuAD_Model = KorQuAD_Model.to(device)
        #optimizer = torch.optim.SGD(mymodel.parameters(), lr=0.0001)
        optimizer = torch.optim.SGD(KorQuAD_Model.parameters(), lr=learning_rate)
        optimizer.zero_grad()
        KorQuAD_Model.train()
        loss = KorQuAD_Model(input_ids, segment_ids, input_mask, start_positions, end_positions)
        loss.backward()
        optimizer.step()
        train_meanloss+=loss
        if (i+1)%100==0:
            print("epoch:{}, step:{}, loss:{}".format(epoch+1,i+1,loss))
    train_meanloss=train_meanloss/i
    train_meanloss=train_meanloss.item() 
    print("epoch:{}  , train mean loss:{}".format((epoch+1),train_meanloss))
    test_meanloss=0
    for j,(input_ids, segment_ids, input_mask, start_positions, end_positions) in enumerate(test_dataloader):
        with torch.no_grad():
            input_ids = torch.tensor(input_ids,dtype=torch.long)
            input_ids = input_ids.to(device)
            input_mask = torch.tensor(input_mask,dtype=torch.long)
            input_mask = input_mask.to(device)
            segment_ids = torch.tensor(segment_ids,dtype=torch.long)
            segment_ids = segment_ids.to(device)
            start_positions = torch.tensor(start_positions,dtype=torch.long)
            start_positions = start_positions.to(device)
            end_positions = torch.tensor(end_positions,dtype=torch.long)
            
            end_positions = end_positions.to(device)
            KorQuAD_Model.eval()
            test_loss = KorQuAD_Model(input_ids, segment_ids, input_mask, start_positions, end_positions)
            test_meanloss+=test_loss.item()
    test_meanloss=test_meanloss/j
#     test_meanloss=test_meanloss.item()
    print("epoch:{}  , test mean loss:{}".format((epoch+1),test_meanloss))
    torch.save({'epoch':epoch,
                'model_state_dict':KorQuAD_Model.state_dict(),
                'optimizer_state_dict':optimizer.state_dict(),
                'loss':loss
               },"/data/KorQuad_saved_models/KQA_MulBert_lr{}_bs{}_ep{}_tl{}.ckpt".format(learning_rate,batch_size,epoch+1,test_meanloss))   
    

epoch:1, step:100, loss:6.222562313079834
epoch:1, step:200, loss:6.020942211151123
epoch:1, step:300, loss:5.925327301025391
epoch:1, step:400, loss:5.981278419494629
epoch:1, step:500, loss:5.670592308044434
epoch:1, step:600, loss:5.611647605895996
epoch:1, step:700, loss:5.4933762550354
epoch:1, step:800, loss:5.36922550201416
epoch:1, step:900, loss:5.401523590087891
epoch:1, step:1000, loss:5.27094841003418
epoch:1, step:1100, loss:5.1262383460998535
epoch:1, step:1200, loss:5.368454933166504
epoch:1, step:1300, loss:4.974310874938965
epoch:1, step:1400, loss:5.06218147277832
epoch:1, step:1500, loss:4.964707374572754
epoch:1, step:1600, loss:5.048808574676514
epoch:1, step:1700, loss:4.812286376953125
epoch:1, step:1800, loss:4.9892258644104
epoch:1, step:1900, loss:4.633874893188477
epoch:1, step:2000, loss:4.759974479675293
epoch:1, step:2100, loss:4.480026721954346
epoch:1, step:2200, loss:4.472637176513672
epoch:1, step:2300, loss:4.794617652893066
epoch:1, step:2400, loss:4

In [ ]:
result_dataloader = DataLoader(test_data,batch_size=1,shuffle=True,num_workers=4)

In [ ]:
em=0
for j,(input_ids, segment_ids, input_mask, start_positions, end_positions) in enumerate(result_dataloader):
    with torch.no_grad():
        input_ids = torch.tensor(input_ids,dtype=torch.long)
        input_ids = input_ids.to(device)
        input_mask = torch.tensor(input_mask,dtype=torch.long)
        input_mask = input_mask.to(device)
        segment_ids = torch.tensor(segment_ids,dtype=torch.long)
        segment_ids = segment_ids.to(device)
        start_positions = torch.tensor(start_positions,dtype=torch.long)
        start_positions = start_positions.to(device)
        end_positions = torch.tensor(end_positions,dtype=torch.long)
        end_positions = end_positions.to(device)
        KorQuAD_Model.eval()
        logit = KorQuAD_Model(input_ids, segment_ids, input_mask)
        
#         print(torch.argmax(logit[0],dim=-1))
        
#         print(start_positions)
        
#         print(torch.argmax(logit[0],dim=-1)==start_positions)
# #         print((torch.argmax(logit[0],dim=-1)==start_positions))
#         print(torch.argmax(logit[1],dim=-1)==end_positions)

#         print((torch.argmax(logit[0],dim=-1)==start_positions) & (torch.argmax(logit[1],dim=-1)==end_positions) )    
    
#         print(torch.sum((torch.argmax(logit[0],dim=-1)==start_positions) & (torch.argmax(logit[1],dim=-1)==end_positions)).item())
        
# #         print((torch.argmax(logit,dim=-1)))
#         em+=torch.sum((torch.argmax(logit[0],dim=-1)==start_positions) & (torch.argmax(logit[1],dim=-1)==end_positions)).item()
        
#         if j>-1:
#             break
        
        if((torch.argmax(logit[0])==start_positions) and (torch.argmax(logit[1])==end_positions)):
            em+=1
#             print(em/j)
#         if(torch.argmax(logit[0])==start_positions):
#             em+=1
#             print(em/j+1)
#         if(torch.argmax(logit[1])==end_positions):
#             em+=1
#             print(em/j+1)

In [ ]:
len(test_data)

In [ ]:
em/len(test_data)

########################################

In [ ]:
checkpoint = torch.load("/data/KorQuad_saved_models/model_ep1.ckpt")

In [40]:
idx=5
sample_input_ids = torch.tensor(test_data[idx:idx+1][0],dtype=torch.long)
sample_input_ids = sample_input_ids.to(device)
sample_input_mask = torch.tensor(test_data[idx:idx+1][1],dtype=torch.long)
sample_input_mask = sample_input_mask.to(device)
sample_segment_ids = torch.tensor(test_data[idx:idx+1][2],dtype=torch.long)
sample_segment_ids = sample_segment_ids.to(device)
sample_start = torch.tensor(test_data[idx:idx+1][3],dtype=torch.long)
sample_start = sample_start.to(device)
sample_end = torch.tensor(test_data[idx:idx+1][4],dtype=torch.long)
sample_end = sample_end.to(device)

In [41]:
start_positions=torch.argmax(KorQuAD_Model(sample_input_ids,sample_input_mask,sample_segment_ids)[0])
start_positions

tensor(119, device='cuda:0')

In [42]:
end_positions=torch.argmax(KorQuAD_Model(sample_input_ids,sample_input_mask,sample_segment_ids)[1])
end_positions

tensor(121, device='cuda:0')

In [43]:
tokenizer.decode(sample_input_ids.squeeze().tolist())

'[CLS] 1978년에 벌어진 노폐물을 감방에 그대로 배출하던 투쟁을 이르는 말은?[SEP] 아일랜드 단식투쟁(아일랜드어: Stailc ocrais 스탈크 오크라스, 영어: Irish hunger strike)은 북아일랜드 분쟁이 진행 중이던 1981년, 북아일랜드의 감옥에 수감되어 있던 아일랜드 공화주의자들이 자신들을 일반 죄수가 아닌 양심수로서 대우해 달라고 요구하며 벌인 단식투쟁이다. 1976년에 모포투쟁으로 시작된 5년간의 양심수 대우 요구 저항 행동의 절정을 장식한 사건이다. 1978년에는 수감자가 감방 밖으로 나가기를 거부하며 배설물을 비롯한 노폐물을 감방에 그대로 배출하는 이른바 불결투쟁이 벌어졌다. 그런데도 영국 정부가 자신들의 요구를 들어줄 기미가 보이지 않자 마침내 1980년, 열 명의 수감자가 제1차 단식투쟁에 돌입해 53일간 단식했다.[SEP] ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇

In [44]:
print ("Gound Truth : {}\nModel Prediction : {}".format(
    tokenizer.decode(sample_input_ids.squeeze().tolist()[sample_start:sample_end+1]),
    tokenizer.decode(sample_input_ids.squeeze().tolist()[start_positions:end_positions+1]))
      )


Gound Truth : 불결투쟁
Model Prediction : 1976년에


In [34]:
KorQuAD_Model.load_state_dict(checkpoint['model_state_dict'])

NameError: name 'checkpoint' is not defined

In [55]:
KorQuAD_Model.state_dict()['qa_outputs.weight']

tensor([[-0.0309,  0.0030,  0.0105,  ...,  0.0164, -0.0319,  0.0018],
        [-0.0260, -0.0251,  0.0266,  ..., -0.0160, -0.0336, -0.0357]],
       device='cuda:1')

In [56]:
checkpoint2 = torch.load("/data/KorQuad_saved_models/model_ep10.ckpt")

In [57]:
KorQuAD_Model.load_state_dict(checkpoint2['model_state_dict'])

<All keys matched successfully>

In [58]:
KorQuAD_Model.state_dict()['qa_outputs.weight']

tensor([[-0.0310,  0.0029,  0.0100,  ...,  0.0168, -0.0317,  0.0016],
        [-0.0259, -0.0253,  0.0265,  ..., -0.0155, -0.0337, -0.0355]],
       device='cuda:1')

In [7]:

for epoch in range(1000):
    meanloss=0
    for i,(input_ids, segment_ids, input_mask, start_positions, end_positions) in enumerate(dataloader):
        input_ids = torch.tensor(input_ids,dtype=torch.long)
        input_ids = input_ids.cuda()
        input_mask = torch.tensor(input_mask,dtype=torch.long)
        input_mask = input_mask.cuda()
        segment_ids = torch.tensor(segment_ids,dtype=torch.long)
        segment_ids = segment_ids.cuda()
        start_positions = torch.tensor(start_positions,dtype=torch.long)
        start_positions = start_positions.cuda()
        end_positions = torch.tensor(end_positions,dtype=torch.long)
        end_positions = end_positions.cuda()
#         KorQuAD_Model = KorQuAD_Model.to(device)
    
        #optimizer = torch.optim.SGD(mymodel.parameters(), lr=0.0001)
        optimizer = torch.optim.SGD(KorQuAD_Model.parameters(), lr=5e-5)
        optimizer.zero_grad()
        loss = KorQuAD_Model(input_ids, segment_ids, input_mask, start_positions, end_positions)
        if (i+1)%100==0:
            print(loss)
        loss.backward()
        optimizer.step()
        meanloss+=loss
    meanloss=meanloss/i
    meanloss=meanloss.item()
    if (epoch+1)%10==0:
        print("epoch:%d  ,   loss:%f"%((epoch+1),meanloss))

/home/byounggeon_kim/anaconda3/envs/test1/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/home/byounggeon_kim/anaconda3/envs/test1/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/byounggeon_kim/anaconda3/envs/test1/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/byounggeon_kim/anaconda3/envs/test1/lib/python3.6/site-packages/ipykernel_l

StopIteration: Caught StopIteration in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/byounggeon_kim/.local/lib/python3.6/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/home/byounggeon_kim/.local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/byounggeon_kim/2021Project/korean-t5/KQA_Model.py", line 17, in forward
    sequence_output, _ = self.bert(input_ids,attention_mask,token_type_ids)
  File "/home/byounggeon_kim/.local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/byounggeon_kim/anaconda3/envs/test1/lib/python3.6/site-packages/transformers/modeling_bert.py", line 734, in forward
    extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
Sto

In [12]:
#########################TEST



In [17]:

test_data[0][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [15]:
from transformers import BertModel

bert = BertModel.from_pretrained("bert-base-multilingual-cased")

In [24]:
output=bert(input_ids=test_data[0][0].unsqueeze(0),attention_mask=test_data[0][1].unsqueeze(0),token_type_ids=test_data[0][2].unsqueeze(0))

In [25]:
output[0].shape

torch.Size([1, 512, 768])

In [26]:
output[1].shape

torch.Size([1, 768])

In [22]:
test_data[0][0].unsqueeze(0)

tensor([[   101,  10004,  17196,  46150,  12638,   9745,  25387,   9670,  18622,
           9751,  21386,  10530,   8853,  31720,  11102,   9645,  35963,   9773,
          11102,   9372,  21928,  10892,    136,    102,   9095,  12030,    113,
           4506,   2179,    114,   9632,  59906,  67074,   9372,  21928,  11467,
          10250,  47955,   9694, 119145,   9405,  67527,  46150,   9694,   9487,
          18623,   9435,  28143,  12030,   8935, 119449,  14279,    117,   9224,
          17138, 118888,    117,   9638,  21386,  14523,  36322,   9568,  33188,
          13374,   8881,  17138,  12609,    119,  41195,  98068,   9574,  37004,
           9281,  42815,  12030,   9638,  65649,    117,   9678, 106589,   9297,
          53529,    117,   9638,   9320,  35979,   8857,  79599,   9744,  12945,
         109522,  12030,   9425,  31720, 118782,   9297,  12030,  27023,  12092,
           8843, 105462,  13374,    117,   9638,  25258,   9568,  33188,  10892,
           9953,  25486,  10